In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
from lyricsgenius import Genius

In [ ]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
genius = Genius(API_KEY)
genius.remove_section_headers = True


In [ ]:
df = pd.read_csv('Hot 100.csv')

In [ ]:
df['chart_date'] = pd.to_datetime(df.chart_date)
df['chart_debut'] = pd.to_datetime(df.chart_debut)

In [ ]:
df['consecutive_weeks'].fillna(value=0,inplace=True)
df['previous_week'].fillna(value=0,inplace=True)

In [ ]:
frame = pd.DataFrame(df.dtypes)
numericals = list(frame.loc[frame.iloc[:,0]=='float64'].iloc[:,0].keys())
numericals = numericals + list(frame.loc[frame.iloc[:,0]=='int64'].iloc[:,0].keys())
for x in numericals:
    df[x] = df[x].astype('int32')

In [ ]:
df.sort_values(by='chart_date',inplace=True)
df.to_pickle('Hot100')

In [ ]:
df.info()

In [ ]:

unique_df = df.drop_duplicates(subset=['song_id'])

In [ ]:
first_pass_df = pd.read_pickle('./complete_df.pkl')
missing_df = first_pass_df.loc[first_pass_df.lyrics.isna()]
missing_df = missing_df.merge(unique_df,how='left',on='song_id')
missing_df.info()

In [ ]:
unique_df.columns

In [ ]:
fill_df = pd.DataFrame()

In [ ]:
fill_df['chart_position'] = missing_df['chart_position_y']
fill_df['chart_date'] = missing_df['chart_date_y']
fill_df['song'] = missing_df['song_y']
fill_df['performer'] = missing_df['performer_y']
fill_df['song_id'] = missing_df['song_id']
fill_df['instance'] = missing_df['instance_y']
fill_df['time_on_chart'] = missing_df['time_on_chart_y']
fill_df['consecutive_weeks'] = missing_df['consecutive_weeks_y']
fill_df['previous_week'] = missing_df['previous_week_y']
fill_df['peak_position'] = missing_df['peak_position_y']
fill_df['worst_position'] = missing_df['worst_position_y']
fill_df['chart_debut'] = missing_df['chart_debut_y']
fill_df['chart_url'] = missing_df['chart_url_y']
fill_df['lyrics'] = missing_df['lyrics']
fill_df.info()

In [ ]:
def get_lyrics_genius(row):
    ans = float('nan')
    title = row['song']
    artist = row['performer']
    try:
        song = genius.search_song(title, artist=artist)
    except:
        ans = float('nan')
    try:
        song.lyrics
    except:
        ans = float('nan')
    else:
        if song.lyrics[:len(title)]==title:
            ans = song.lyrics[len(title)+len(' lyrics'):]
        else:
            ans = float('nan')
    return(ans)


In [ ]:
fill_df['lyrics'] = fill_df.apply(get_lyrics_genius,axis=1)

In [ ]:
done_df = fill_df.loc[fill_df.lyrics.notna()]
not_done_df = fill_df.loc[fill_df.lyrics.isna()]

In [ ]:
done_df.info()

In [ ]:
not_done_df.info()

In [ ]:
done_df.to_pickle('./2ndpassdataframe.pkl')

In [ ]:
not_done_df.to_pickle('./3rdpassdataframe.pkl')